In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

In [ ]:
# Go to this folder: https://drive.google.com/drive/folders/1MbxJkTyZX-S9jZYYC-BWK4yeFqVBW-Mf?usp=drive_link
# Use the files final_features and final_outcomes

df_features = pd.read_csv("features_final_v2.csv")
df_outcomes = pd.read_csv("final_outcomes.csv")

median_los = df_outcomes['length_of_stay'].median()
df_outcomes['prolonged_length_of_stay'] = (df_outcomes['length_of_stay'] > median_los).astype(int)

In [ ]:
df_outcomes

In [ ]:
df_outcomes['aki_stage'].value_counts()

In [ ]:
df_outcomes['length_of_stay'].median()

In [ ]:
df_outcomes['aki_status'].value_counts()

# Feature List

In [ ]:
features = {
    'length_of_stay_use_apache': [
        'age',
        'has_heartdisease',
        'has_diabetes',
        'has_cancer',
        'has_esrf',
        'apache_temperature',
        'apache_heartrate',
        'apache_fio2',
        'systolic_noninvasive',
        'systolic_invasive',
        'wbc1000',
        'sodium',
        'potassium',
        'apache_gcs',
        'bilirubin',
        'bicarbonate'
    ],
    'length_of_stay': [
        'age',
        'has_heartdisease',
        'has_diabetes',
        'has_cancer',
        'has_esrf',
        'apache_temperature',
        'temperature',
        'apache_heartrate',
        'heartrate',
        'pao2_fio2_ratio',
        'systolic_noninvasive',
        'systolic_invasive',
        'wbc1000',
        'sodium',
        'potassium',
        'apache_gcs',
        'bilirubin',
        'bicarbonate'
    ],
    'mortality': [
        # paper for the features: https://pmc.ncbi.nlm.nih.gov/articles/PMC7332047
        # - We made some minor modifications with regards to these features:
        # 1. Apacheadmissiondx: This can be quite vague so would need sufficient cleaning?
        # 2. nursingChartOffset: Not sure what the relevance of this timestamp is?
        # 3. Unitdischargeoffset: This doesn't make sense to use because it won't be available at the time of prediction, unless we were predicting based on the last values during the stay.
        # 4. eyes / motor / verbal: These three values are already captured in the GCS, so there is no need to include it as additional variables.
        # 5. invasive sys/dia/map: I swapped these out with noninvasive measures, as there are more samples (not everyone would be measured invasively) and should still approximate to the same values.
        'age',
        'ethnicity',
        'gender',
        'height',
        'weight',
        'sao2',
        'systolic_noninvasive',
        'diastolic_noninvasive',
        'map_noninvasive',
        'apache_heartrate',
        'apache_respiratoryrate',
        'apache_temperature',
        'apache_glucose',
        'apache_fio2',
        'apache_ph',
        'apache_gcs'
    ],
    'aki': [
        # This was loosely based off this paper: https://www.nature.com/articles/s41598-021-97735-0
        # - We did not include the following features, because there are some disagreements around how accurately these values are captured
        # in the dataset, because not everyone may be on a catheter, and the window with which you calculate it, par around the idea of urine output. If performance is very
        # low, we could always attempt to re-calculate it.
        # 1. Hourly measures of urine output
        # 2. Fluid intake
        # 3. Fluid Balance (computed as fluid input minus output, normalized by weight)
        'age',
        'gender',
        'weight',
        'height',
        'lean_body_mass',
        'has_heartdisease',
        'has_diabetes',
        'has_cancer',
        'vasopressin_used',
        'diuretics_used',
        'creatinine',
        'map_noninvasive',
    ],
    # paper: https://pubmed.ncbi.nlm.nih.gov/36653875/
    # new paper: https://www.nature.com/articles/s41598-024-63793-3
    'aki_v2': [
        'gender',
        'age',
        'sao2',
        'apache_ph',
        'apache_temperature',
        'apache_heartrate',
        'apache_respiratoryrate',
        'apache_gcs',
        'apache_fio2',
        'apache_glucose',
        'apache_bun',
        'apache_creatinine',
        'apache_wbc',
        'apache_sodium',
        'apache_hematocrit',
        'apache_pco2',
        'apache_pao2',
        'bicarbonate',
        'potassium',
        'chloride',
        'hemoglobin',
        'platelet_count'
    ]
}



In [ ]:
df_features.columns.to_list()

## Preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline

# Taken from PSET 1
# Drop rows with any missing values

# For numerical features (no need for an imputer now)
numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler())  # Scale numerical data
    ]
)

# For categorical features (no need for an imputer now)
categorical_transformer = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"))  # One-hot encode categorical data
    ]
)

# Setting up the ColumnTransformer
preprocessing = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude=["category", object, "string"])),
        ("cat", categorical_transformer, selector(dtype_include=["category", object, "string"]))
    ]
)

# Complete preprocessing pipeline
preprocessing_pipeline = Pipeline(
    steps=[("preprocessing", preprocessing)]
)


In [ ]:
# For numerical features

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),  # Impute missing values with mean
        ("scaler", StandardScaler())  # Scale numerical data
    ]
)

# For categorical features
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),  # Impute missing values with the most frequent value
        ("onehot", OneHotEncoder(handle_unknown="ignore"))  # One-hot encode categorical data
    ]
)

preprocessing = ColumnTransformer(
    transformers=[
        (
            "num",
            numeric_transformer,
            selector(dtype_exclude=["category", object, "string"]),
        ),
        (
            "cat",
            categorical_transformer,
            selector(dtype_include=["category", object, "string"]),
        ),
    ]
)

preprocessing_pipeline_v2 = Pipeline(
    steps=[("preprocessing", preprocessing)]
)

In [ ]:
def preprocess(df, df_out, features, outcomes, impute=False):
    # Drop rows with any NaNs in either DataFrame

    if not impute:
      df = df.dropna(subset=features).reset_index(drop=True)
    df_out = df_out.dropna(subset=outcomes).reset_index(drop=True)

    # Ensure 'uniquepid' is included in features
    if 'uniquepid' not in features:
        features.append('uniquepid')

    # Select specified features from df
    X = df[features]

    # Filter df_out to include only those rows corresponding to X's 'uniquepid'
    Y_df = df_out[df_out['uniquepid'].isin(X['uniquepid'])]

    Y = pd.DataFrame()
    for outcome in outcomes:
        Y[outcome] = pd.to_numeric(Y_df[outcome], errors='coerce')

    # Drop any rows where Y is NaN (failed conversion to numeric)
    valid_indices = Y.dropna().index

    # Get valid 'uniquepid's
    valid_uniquepids = Y_df.loc[valid_indices, 'uniquepid']

    # Align X to only include rows with valid 'uniquepid's
    X_with_pid = X[X['uniquepid'].isin(valid_uniquepids)].reset_index(drop=True)
    Y = Y.loc[valid_indices].reset_index(drop=True)
    unique_pids = X_with_pid['uniquepid'].reset_index(drop=True)
    X_with_pid = X_with_pid.drop(columns=['uniquepid'])

    # Apply the preprocessing pipeline
    if not impute:
      X_processed = preprocessing_pipeline.fit_transform(X_with_pid)
    else:
      X_processed = preprocessing_pipeline_v2.fit_transform(X_with_pid)

    return X_processed, Y, unique_pids

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate_model(y_pred_proba, y_test, threshold=0.5):
    # Convert probabilities to class labels based on threshold
    y_pred_labels = (y_pred_proba >= threshold).astype(int)

    accuracy = accuracy_score(y_test, y_pred_labels)
    precision = precision_score(y_test, y_pred_labels, zero_division=0)
    recall = recall_score(y_test, y_pred_labels, zero_division=0)
    f1 = f1_score(y_test, y_pred_labels, zero_division=0)
    auc = roc_auc_score(y_test, y_pred_proba)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC Score: {auc:.4f}")

#Models

##XGBoost

In [ ]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix  # For evaluation

def train_and_evaluate_xgboost(X, Y, uniquepid, outcome):
    print("Training XGBoost")
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test, pid_train, pid_test = train_test_split(
        X, Y, uniquepid, test_size=0.2, random_state=42
    )

    # Creating and fitting the XGBoost model
    xgb_model = xgboost.XGBClassifier(random_state=42, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = xgb_model.predict(X_test)

    # Predicting probabilities on the test set
    y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Predicting class labels based on a threshold (default 0.5)
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Evaluation
    evaluate_model(y_pred_proba, y_test)

    # Evaluation
    # accuracy = accuracy_score(y_test, y_pred)
    # print(f"Accuracy: {accuracy:.4f}")
    # cm = confusion_matrix(y_test, y_pred)
    # print("Confusion Matrix:")
    # print(cm)

    # evaluate_model(y_pred, y_test)

    # Creating a DataFrame with true and predicted values along with uniquepid
    outcome_df = pd.DataFrame({
        'uniquepid': pid_test.reset_index(drop=True),
        f'actual_{outcome}': y_test.reset_index(drop=True),
        f'predicted_{outcome}': y_pred
    })

    return outcome_df, xgb_model


def train_and_evaluate_xgboost_v2(X_train, X_test, y_train, y_test, pid_train, pid_test, outcome):
    print("Training XGBoost")

    # Creating and fitting the XGBoost model
    xgb_model = xgboost.XGBClassifier(random_state=42, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = xgb_model.predict(X_test)

    # Predicting probabilities on the test set
    y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Predicting class labels based on a threshold (default 0.5)
    y_pred = (y_pred_proba >= 0.5).astype(int)

    print('xgboost', len(y_pred))

    # Evaluation
    evaluate_model(y_pred_proba, y_test)

    # Evaluation
    # accuracy = accuracy_score(y_test, y_pred)
    # print(f"Accuracy: {accuracy:.4f}")
    # cm = confusion_matrix(y_test, y_pred)
    # print("Confusion Matrix:")
    # print(cm)

    # evaluate_model(y_pred, y_test)

    # Creating a DataFrame with true and predicted values along with uniquepid
    outcome_df = pd.DataFrame({
        'uniquepid': pid_test,
        f'actual_{outcome}': y_test,
        f'predicted_{outcome}': y_pred
    })

    return outcome_df, xgb_model

##Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

def train_and_evaluate_logistic_regression(X, Y, uniquepid, outcome):
    print("Training Logistic Regression")
    # Splitting the data
    X_train, X_test, y_train, y_test, pid_train, pid_test = train_test_split(
        X, Y, uniquepid, test_size=0.2, random_state=42
    )

    # Creating and fitting the model
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train, y_train)

    # Predicting probabilities on the test set
    y_pred_proba = lr_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Predicting class labels based on a threshold
    y_pred = (y_pred_proba >= 0.5).astype(int)

    # Evaluation
    evaluate_model(y_pred_proba, y_test)

    # Outcome DataFrame
    outcome_df = pd.DataFrame({
        'uniquepid': pid_test.reset_index(drop=True),
        f'actual_{outcome}': y_test.reset_index(drop=True),
        f'predicted_{outcome}': y_pred
    })

    return outcome_df, lr_model


def train_and_evaluate_logistic_regression_v2(X_train, X_test, y_train, y_test, pid_train, pid_test, outcome):
    print("Training Logistic Regression")
    # Splitting the data
    # X_train, X_test, y_train, y_test, pid_train, pid_test = train_test_split(
    #     X, Y, uniquepid, test_size=0.2, random_state=42
    # )

    # Creating and fitting the model
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train, y_train)

    # Predicting probabilities on the test set
    y_pred_proba = lr_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Predicting class labels based on a threshold
    y_pred = (y_pred_proba >= 0.5).astype(int)
    print('logreg', len(y_pred))

    # Evaluation
    evaluate_model(y_pred_proba, y_test)

    # Outcome DataFrame
    outcome_df = pd.DataFrame({
        'uniquepid': pid_test,
        f'actual_{outcome}': y_test,
        f'predicted_{outcome}': y_pred
    })

    return outcome_df, lr_model

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def train_and_evaluate_random_forest(X, Y, uniquepid, outcome):
    print("Training Random Forest")
    # Splitting the data
    X_train, X_test, y_train, y_test, pid_train, pid_test = train_test_split(
        X, Y, uniquepid, test_size=0.2, random_state=42
    )

    # Creating and fitting the model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = rf_model.predict(X_test)

    # Predicting probabilities on the test set
    y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

    # Predicting class labels based on a threshold
    y_pred = (y_pred_proba >= 0.5).astype(int)
    print('rf', len(y_pred))

    # Evaluation
    evaluate_model(y_pred_proba, y_test)

    # Outcome DataFrame
    outcome_df = pd.DataFrame({
        'uniquepid': pid_test.reset_index(drop=True),
        f'actual_{outcome}': y_test.reset_index(drop=True),
        f'predicted_{outcome}': y_pred
    })

    return outcome_df, rf_model

def train_and_evaluate_random_forest_v2(X_train, X_test, y_train, y_test, pid_train, pid_test, outcome):
    print("Training Random Forest")

    # Creating and fitting the Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)

    # Predicting probabilities on the test set
    y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Predicting class labels based on a threshold (default 0.5)
    y_pred = (y_pred_proba >= 0.5).astype(int)

    print('random_forest', len(y_pred))

    # Evaluation
    evaluate_model(y_pred_proba, y_test)

    # Creating a DataFrame with true and predicted values along with uniquepid
    outcome_df = pd.DataFrame({
        'uniquepid': pid_test.reset_index(drop=True),
        f'actual_{outcome}': y_test.reset_index(drop=True),
        f'predicted_{outcome}': y_pred
    })

    return outcome_df, rf_model


In [ ]:
all_features = list(set(features['length_of_stay'] + features['mortality'] + features['aki_v2']))
outcomes = ['prolonged_length_of_stay', 'aki_status', 'hospital_mortality']
feature_df = df_features
X, Y, uniquepid, feature_list = preprocess(feature_df, df_outcomes, all_features, outcomes, True)

In [ ]:
feature_list

# Run Analysis

Uses selected features for each task only. Patients with nan values are dropped on a case by case basis based on the prediction tasks and the features necessary for that prediction task.

In [ ]:
def run_analysis(feature_list, outcome, feature_df):
  X, Y, uniquepid = preprocess(feature_df, df_outcomes, feature_list, outcome)

  print('-----------------------------------')
  outcome1_df, xgb_model = train_and_evaluate_xgboost(X, Y, uniquepid, outcome)
  print('-----------------------------------')
  outcome2_df, lr_model = train_and_evaluate_logistic_regression(X, Y, uniquepid, outcome)
  print('-----------------------------------')
  outcome3_df, rf_model = train_and_evaluate_random_forest(X, Y, uniquepid, outcome)
  print('-----------------------------------')


  outcome1_df = outcome1_df.rename(columns={f'predicted_{outcome}': f'predicted_{outcome}_xgb'})
  outcome2_df = outcome2_df.rename(columns={f'predicted_{outcome}': f'predicted_{outcome}_lr'})
  outcome3_df = outcome3_df.rename(columns={f'predicted_{outcome}': f'predicted_{outcome}_rf'})

  merged_df = outcome1_df.merge(
      outcome2_df[['uniquepid', f'predicted_{outcome}_lr']],
      on='uniquepid'
  ).merge(
      outcome3_df[['uniquepid', f'predicted_{outcome}_rf']],
      on='uniquepid'
  )

  merged_df.to_csv(f'{outcome}_results.csv', index=False)


def run_analysis_v2(feature_list, outcome, feature_df, df_outcomes, outcomes, impute):
    # Preprocess the data
    X, Y, uniquepid = preprocess(feature_df, df_outcomes, feature_list, outcomes, impute)

    pd.DataFrame(X).to_csv('overall_train_test_features.csv')
    pd.DataFrame(Y).to_csv('overall_train_test_outcomes.csv')


    # Split the data
    X_train, X_test, y_train, y_test, pid_train, pid_test = train_test_split(
        X, Y[outcome], uniquepid, test_size=0.2, random_state=42
    )


    # # Debugging: Check types and lengths
    # print("Type of X_train:", type(X_train))
    # print("Type of pid_train:", type(pid_train))
    # print("Type of y_train:", type(y_train))
    # print("Length of X_train:", len(X_train))
    # print("Length of pid_train:", len(pid_train))
    # print("Length of y_train:", len(y_train))

    # # Ensure X_train and X_test are DataFrames
    # if not isinstance(X_train, pd.DataFrame):
    #     X_train = pd.DataFrame(X_train)
    # if not isinstance(X_test, pd.DataFrame):
    #     X_test = pd.DataFrame(X_test)

    # # Ensure pid_train and pid_test are Series
    # if not isinstance(pid_train, pd.Series):
    #     pid_train = pd.Series(pid_train)
    # if not isinstance(pid_test, pd.Series):
    #     pid_test = pd.Series(pid_test)

    # # Ensure y_train and y_test are DataFrames
    # if isinstance(y_train, pd.Series):
    #     y_train = y_train.to_frame(name=outcome)
    # if isinstance(y_test, pd.Series):
    #     y_test = y_test.to_frame(name=outcome)

    # # Assign 'uniquepid' to X_train and X_test
    # try:
    #     X_train = X_train.reset_index(drop=True)
    #     X_test = X_test.reset_index(drop=True)
    #     pid_train = pid_train.reset_index(drop=True)
    #     pid_test = pid_test.reset_index(drop=True)

    #     X_train['uniquepid'] = pid_train
    #     X_test['uniquepid'] = pid_test
    # except Exception as e:
    #     print(f"Error during assigning 'uniquepid' to X_train/X_test: {e}")
    #     print("X_train columns before assignment:", X_train.columns)
    #     print("pid_train head:", pid_train.head())
    #     return

    # # Assign 'uniquepid' to y_train and y_test
    # try:
    #     y_train['uniquepid'] = pid_train
    #     y_test['uniquepid'] = pid_test
    # except Exception as e:
    #     print(f"Error during assigning 'uniquepid' to y_train/y_test: {e}")
    #     print("y_train columns before assignment:", y_train.columns)
    #     print("pid_train head:", pid_train.head())
    #     return

    # # Debugging: Verify assignments
    # print("X_train columns after assignment:", X_train.columns)
    # print("y_train columns after assignment:", y_train.columns)

    # # Exporting DataFrames to CSV
    # X_train.to_csv('training_set_features_df.csv', index=False)
    # X_test.to_csv('test_set_features_df.csv', index=False)
    # y_train.to_csv('training_set_outcomes_df.csv', index=False)
    # y_test.to_csv('test_set_outcomes_df.csv', index=False)

    print('-----------------------------------')

    # Train and evaluate models
    outcome1_df, xgb_model = train_and_evaluate_xgboost_v2(X_train, X_test, y_train, y_test, pid_train, pid_test, outcome)
    print('-----------------------------------')
    outcome2_df, lr_model = train_and_evaluate_logistic_regression_v2(X_train, X_test, y_train, y_test, pid_train, pid_test, outcome)
    print('-----------------------------------')
    outcome3_df, rf_model = train_and_evaluate_random_forest_v2(X_train, X_test, y_train, y_test, pid_train, pid_test, outcome)
    print('-----------------------------------')

    # Rename prediction columns
    outcome1_df = outcome1_df.rename(columns={f'predicted_{outcome}': f'predicted_{outcome}_xgb'})
    outcome2_df = outcome2_df.rename(columns={f'predicted_{outcome}': f'predicted_{outcome}_lr'})
    outcome3_df = outcome3_df.rename(columns={f'predicted_{outcome}': f'predicted_{outcome}_rf'})

    # Merge predictions on 'uniquepid'
    merged_df = outcome1_df.merge(
        outcome2_df[['uniquepid', f'predicted_{outcome}_lr']],
        on='uniquepid'
    ).merge(
        outcome3_df[['uniquepid', f'predicted_{outcome}_rf']],
        on='uniquepid'
    )

    # Export merged predictions to CSV
    merged_df.to_csv(f'{outcome}_results.csv', index=False)

In [ ]:
print('Prolonged Length of Stay')

run_analysis(features['length_of_stay'], 'prolonged_length_of_stay', df_features)
print('Mortality')
run_analysis(features['mortality'], 'hospital_mortality', df_features)
print('AKI')
run_analysis(features['aki_v2'], 'aki_status', df_features)

# Standardize Training Set

Basically concatenate features for all tasks, and run analysis on that dataframe. That way the same patients get dropped for having nan values, and we have a standardized training and test set across all tasks. This should make analysis easier.

In [ ]:
all_features = list(set(features['length_of_stay'] + features['mortality'] + features['aki_v2'] + ['uniquepid']))
outcomes = list(['aki_status', 'prolonged_length_of_stay', 'hospital_mortality'])

run_analysis_v2(all_features, 'hospital_mortality', df_features, df_outcomes, outcomes, impute=True)
run_analysis_v2(all_features, 'aki_status', df_features, df_outcomes, outcomes, impute=True)
run_analysis_v2(all_features, 'prolonged_length_of_stay', df_features, df_outcomes, outcomes, impute=True)


In [ ]:
num_nans = pid_test.isna().sum()
print(num_nans)

In [ ]:
pid_train.to_csv('trainpid.csv')
pid_test.to_csv('testpid.csv')

# graph

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_demographics(uniquepids, df_features, categorical_columns=['ethnicity', 'gender'], numerical_columns=['age', 'weight']):
    filtered_df = df_features[df_features['uniquepid'].isin(uniquepids)]
    sns.set_theme(style="whitegrid", palette="pastel")
    for column in categorical_columns:
        if column in filtered_df.columns:
            value_counts = filtered_df[column].value_counts()
            # Pie chart
            plt.figure(figsize=(8, 8))
            colors = sns.color_palette("pastel", len(value_counts))
            plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)
            plt.title(f"Distribution of {column.capitalize()} (Pie Chart)", fontsize=14, fontweight='bold')
            plt.show()
            # Bar chart
            plt.figure(figsize=(10, 6))
            sns.barplot(x=value_counts.index, y=value_counts.values, palette=colors)
            plt.title(f"Distribution of {column.capitalize()} (Bar Chart)", fontsize=14, fontweight='bold')
            plt.ylabel("Count", fontsize=12)
            plt.xlabel(column.capitalize(), fontsize=12)
            plt.xticks(rotation=45)
            plt.show()
    for column in numerical_columns:
        if column in filtered_df.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(filtered_df[column].dropna(), bins=20, kde=True, color="skyblue")
            plt.title(f"Distribution of {column.capitalize()}", fontsize=14, fontweight='bold')
            plt.xlabel(column.capitalize(), fontsize=12)
            plt.ylabel("Frequency", fontsize=12)
            plt.grid(True, linestyle="--", alpha=0.6)
            plt.show()

uniquepids = df_features['uniquepid'].unique()
plot_demographics(
    uniquepids=uniquepids,
    df_features=df_features,
    categorical_columns=['ethnicity', 'gender', 'has_diabetes', 'has_heartdisease', 'has_cancer', 'has_esrf'],
    numerical_columns=['age', 'weight', 'height']
)


In [ ]:
df_features['uniquepid'].value_counts()

In [ ]:
len(df_features['uniquepid'].unique()) == len(df_features['uniquepid'])